In [1]:
import math 
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
#url에 접속
html=urlopen('https://movie.naver.com/movie/running/current.nhn?order=reserve')
#html 분석기
bs=BeautifulSoup(html, 'html.parser') 
#태그 수집
items = bs.select('#content > div.article > div > div.lst_wrap > ul > li > dl > dt > a')
codes=[]
titles=[]
for item in items:
    # <a href=''>텍스트</a>  
    # href 속성에서 =을 기준으로 나눔
    code=(item['href'].split('=')[1])
    title=item.text 
    codes.append(code)
    titles.append(title)

In [2]:
import pandas as pd 
#데이터프레임으로 바꾸고
df=pd.DataFrame({'code':codes, 'title':titles})
df.head()

,code,title
0,163834,원더 우먼 1984
1,199393,조제
2,193194,도굴
3,164143,이웃사촌
4,36843,러브 액츄얼리


In [3]:
#csv 파일로 저장
df.to_csv('c:/data/movie/movie_code.csv', encoding='ms949',index=False)

In [4]:
total_reviews=[]
total_points=[]
movie_codes=[]
movie_titles=[]
total_reviews2=[]
labels=[]
for code in codes[:20]:
    #수집할 url
    html=urlopen(f'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={code}&target=after&page=1')
    bs=BeautifulSoup(html, 'html.parser')
    # 태그 1개만 수집 select_one
    total=bs.select_one('#old_content > h5 > div > strong')
    #print(total)
    # 좌우 공백 제거, (), 제거
    #value=total.text.strip().replace('(','').replace(')','').replace(',','')
    pages=int(total.text)/10 
    pages=math.ceil(pages)  #올림
    # print('페이지수:',pages)

    #for i in range(1,pages+1):
    for i in range(1,11):
        # print(f'page {i}')
        url=f'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={code}&target=after&page={i}'
        webpage=urlopen(url)
        source=BeautifulSoup(webpage, 'html.parser')
        #영화리뷰텍스트
        reviews=source.select('tbody > tr > td.title')
        if i>pages: continue 
        for review in reviews:
            total_reviews.append(review) 
        #영화평점
        points=source.select('td.title > div > em')
        for point in points:
            total_points.append(point.text) 
            # 8점 이상이면 1, 아니면 0으로 라벨링
            if int(point.text)>=8:
                labels.append(1)
            else:
                labels.append(0)
    for review in total_reviews:
        #영화코드 추가
        movie_codes.append(code)
        #리뷰 텍스트만 추출
        total_reviews2.append(review.text.strip().split('\n\n')[2].replace('\t','').replace('\n','').replace('네이버','').replace('신고',''))

페이지수: 14
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 149
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 318
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 196
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 557
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 83
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 586
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 108
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 50
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 5
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 3
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
페이지수: 3
page 1
page 2
page 3
page 4
p

In [5]:
#영화코드,리뷰,평점,라벨 을 결합
items=list(zip(movie_codes, total_reviews2, total_points, labels))
print(len(items))

1576


In [6]:
#데이터프레임으로 변환하고 파일로 저장
df=pd.DataFrame(items, columns=['code','review','point','label'])
df.to_csv('c:/data/movie/review.csv', encoding='utf-8', index=False)

In [7]:
#파일에서 읽어오기
df2=pd.read_csv('c:/data/movie/review.csv', encoding='utf-8')
df2.head()

,code,review,point,label
0,163834,good,10,1
1,163834,세상과 아들을 구해요 맥스로드,10,1
2,163834,코로나때문에 답답한마음이 조금이나마 위안을 받았음 욕심과 탐욕 이기심때문에 점점...,10,1
3,163834,이제 그만 욕망을 멈추고 주변을 사랑하라가 이 영화의 메시지인것 같다. 그리고 감독...,10,1
4,163834,1편만큼만해라 하고 별기대없이 관람했는데기대이상으로 흥미 재미가 있습니다.마지막 장...,9,1
